In [ ]:
from flair.data import Corpus
from flair.datasets import UD_ENGLISH
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from flair.embeddings import CharacterEmbeddings
corpus = flair.datasets.IMDB().downsample(0.1)

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

# 4. initialize embeddings
embedding_types = [
    
    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
    
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-pos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=2)

2020-06-27 02:07:57,553 http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz not found in cache, downloading to /var/folders/5m/h_g_92_s1d11s4pd5cbhs48m0000gn/T/tmpj5i_8fv2


100%|██████████| 84125825/84125825 [00:15<00:00, 5410086.70B/s]

2020-06-27 02:08:13,723 copying /var/folders/5m/h_g_92_s1d11s4pd5cbhs48m0000gn/T/tmpj5i_8fv2 to cache at /Users/subir/.flair/datasets/imdb_v2-rebalanced/aclImdb_v1.tar.gz


2020-06-27 02:08:14,000 removing temp file /var/folders/5m/h_g_92_s1d11s4pd5cbhs48m0000gn/T/tmpj5i_8fv2
2020-06-27 02:08:38,777 Reading data from /Users/subir/.flair/datasets/imdb_v2-rebalanced
2020-06-27 02:08:38,777 Train: /Users/subir/.flair/datasets/imdb_v2-rebalanced/train-all.txt
2020-06-27 02:08:38,778 Dev: None
2020-06-27 02:08:38,779 Test: None
Dictionary with 5 tags: <unk>, O, , <START>, <STOP>
2020-06-27 02:09:10,563 ----------------------------------------------------------------------------------------------------
2020-06-27 02:09:10,565 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      

In [13]:
# load the model you trained
model = SequenceTagger.load('resources/taggers/example-pos/final-model.pt')

# create example sentence
sentence = Sentence('I love Berlin')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

2020-06-27 01:51:17,327 loading file resources/taggers/example-pos/final-model.pt
I <PRP> love <VBP> Berlin <NNP>
